In [56]:
%matplotlib inline

from datetime import date
import pandas as pd
import urllib.request
import xmltodict
from ipywidgets import HTML
from ipyleaflet import *
import configparser

In [61]:
config = configparser.ConfigParser()
config.read("config.cfg")

['config.cfg']

In [2]:
import math

axis = None # Semi-major axis of the ellipsoid.
flattening = None # Flattening of the ellipsoid.
central_meridian = None # Central meridian for the projection.
lat_of_origin = None # Latitude of origin.
scale = None # Scale on central meridian.
false_northing = None # Offset for origo.
false_easting = None # Offset for origo.

# Parameters for RT90 and SWEREF99TM.
# Note: Parameters for RT90 are choosen to eliminate the 
# differences between Bessel and GRS80-ellipsoides.
# Bessel-iants should only be used if lat/long are given as
# RT90-lat/long based on the Bessel ellipsoide (from old maps).
# Parameter: projection (string). Must match if-statement.
def swedish_params(projection) :

    global central_meridian 
    global scale 
    global false_northing 
    global false_easting 
    global lat_of_origin

    # RT90 parameters, GRS 80 ellipsoid.
    if (projection == "rt90_7.5_gon_v") :
        grs80_params()
        central_meridian = 11.0 + 18.375/60.0
        scale = 1.000006000000
        false_northing = -667.282
        false_easting = 1500025.141

    elif (projection == "rt90_5.0_gon_v") :
        grs80_params()
        central_meridian = 13.0 + 33.376/60.0
        scale = 1.000005800000
        false_northing = -667.130
        false_easting = 1500044.695

    elif (projection == "rt90_2.5_gon_v") :
        grs80_params()
        central_meridian = 15.0 + 48.0/60.0 + 22.624306/3600.0
        scale = 1.00000561024
        false_northing = -667.711
        false_easting = 1500064.274

    elif (projection == "rt90_0.0_gon_v") :
        grs80_params()
        central_meridian = 18.0 + 3.378/60.0
        scale = 1.000005400000
        false_northing = -668.844
        false_easting = 1500083.521

    elif (projection == "rt90_2.5_gon_o") :
        grs80_params()
        central_meridian = 20.0 + 18.379/60.0
        scale = 1.000005200000
        false_northing = -670.706
        false_easting = 1500102.765

    elif (projection == "rt90_5.0_gon_o") :
        grs80_params()
        central_meridian = 22.0 + 33.380/60.0
        scale = 1.000004900000
        false_northing = -672.557
        false_easting = 1500121.846


    # RT90 parameters, Bessel 1841 ellipsoid.
    elif (projection == "bessel_rt90_7.5_gon_v") :
        bessel_params()
        central_meridian = 11.0 + 18.0/60.0 + 29.8/3600.0

    elif (projection == "bessel_rt90_5.0_gon_v") :
        bessel_params()
        central_meridian = 13.0 + 33.0/60.0 + 29.8/3600.0

    elif (projection == "bessel_rt90_2.5_gon_v") :
        bessel_params()
        central_meridian = 15.0 + 48.0/60.0 + 29.8/3600.0

    elif (projection == "bessel_rt90_0.0_gon_v") :
        bessel_params()
        central_meridian = 18.0 + 3.0/60.0 + 29.8/3600.0

    elif (projection == "bessel_rt90_2.5_gon_o") :
        bessel_params()
        central_meridian = 20.0 + 18.0/60.0 + 29.8/3600.0

    elif (projection == "bessel_rt90_5.0_gon_o") :
        bessel_params()
        central_meridian = 22.0 + 33.0/60.0 + 29.8/3600.0


    # SWEREF99TM and SWEREF99ddmm  parameters.
    elif (projection == "sweref_99_tm") :
        sweref99_params()
        central_meridian = 15.00
        lat_of_origin = 0.0
        scale = 0.9996
        false_northing = 0.0
        false_easting = 500000.0

    elif (projection == "sweref_99_1200") :
        sweref99_params()
        central_meridian = 12.00

    elif (projection == "sweref_99_1330") :
        sweref99_params()
        central_meridian = 13.50

    elif (projection == "sweref_99_1500") :
        sweref99_params()
        central_meridian = 15.00

    elif (projection == "sweref_99_1630") :
        sweref99_params()
        central_meridian = 16.50

    elif (projection == "sweref_99_1800") :
        sweref99_params()
        central_meridian = 18.00

    elif (projection == "sweref_99_1415") :
        sweref99_params()
        central_meridian = 14.25

    elif (projection == "sweref_99_1545") :
        sweref99_params()
        central_meridian = 15.75

    elif (projection == "sweref_99_1715") :
        sweref99_params()
        central_meridian = 17.25

    elif (projection == "sweref_99_1845") :
        sweref99_params()
        central_meridian = 18.75

    elif (projection == "sweref_99_2015") :
        sweref99_params()
        central_meridian = 20.25

    elif (projection == "sweref_99_2145") :
        sweref99_params()
        central_meridian = 21.75

    elif (projection == "sweref_99_2315") :
        sweref99_params()
        central_meridian = 23.25


    # Test-case:
    #  Lat: 66 0'0", lon: 24 0'0".
    #  X:1135809.413803 Y:555304.016555.
    elif (projection == "test_case") :
        axis = 6378137.0
        flattening = 1.0 / 298.257222101
        central_meridian = 13.0 + 35.0/60.0 + 7.692000/3600.0
        lat_of_origin = 0.0
        scale = 1.000002540000
        false_northing = -6226307.8640
        false_easting = 84182.8790

    # Not a valid projection.      
    else :
        central_meridian = None


# Sets of default parameters.
def grs80_params() :

    global axis 
    global flattening 
    global central_meridian 
    global lat_of_origin 

    axis = 6378137.0 # GRS 80.
    flattening = 1.0 / 298.257222101 # GRS 80.
    central_meridian = None
    lat_of_origin = 0.0

def bessel_params() :

    global axis 
    global flattening 
    global central_meridian 
    global lat_of_origin 
    global scale 
    global false_northing 
    global false_easting 

    axis = 6377397.155 # Bessel 1841.
    flattening = 1.0 / 299.1528128 # Bessel 1841.
    central_meridian = None
    lat_of_origin = 0.0
    scale = 1.0
    false_northing = 0.0
    false_easting = 1500000.0

def sweref99_params() :

    global axis 
    global flattening 
    global central_meridian 
    global lat_of_origin 
    global scale 
    global false_northing 
    global false_easting 

    axis = 6378137.0 # GRS 80.
    flattening = 1.0 / 298.257222101 # GRS 80.
    central_meridian = None
    lat_of_origin = 0.0
    scale = 1.0
    false_northing = 0.0
    false_easting = 150000.0


# Conversion from geodetic coordinates to grid coordinates.
def geodetic_to_grid(latitude, longitude) :
    x_y = [0] * 2
    if (central_meridian == None) :
        return x_y

    # Prepare ellipsoid-based stuff.
    e2 = flattening * (2.0 - flattening)
    n = flattening / (2.0 - flattening)
    a_roof = axis / (1.0 + n) * (1.0 + n*n/4.0 + n*n*n*n/64.0)
    A = e2
    B = (5.0*e2*e2 - e2*e2*e2) / 6.0
    C = (104.0*e2*e2*e2 - 45.0*e2*e2*e2*e2) / 120.0
    D = (1237.0*e2*e2*e2*e2) / 1260.0
    beta1 = n/2.0 - 2.0*n*n/3.0 + 5.0*n*n*n/16.0 + 41.0*n*n*n*n/180.0
    beta2 = 13.0*n*n/48.0 - 3.0*n*n*n/5.0 + 557.0*n*n*n*n/1440.0
    beta3 = 61.0*n*n*n/240.0 - 103.0*n*n*n*n/140.0
    beta4 = 49561.0*n*n*n*n/161280.0

    # Convert.
    deg_to_rad = math.pi / 180.0
    phi = latitude * deg_to_rad
    lambda_ = longitude * deg_to_rad
    lambda_zero = central_meridian * deg_to_rad

    phi_star = phi - math.sin(phi) * math.cos(phi) * (A + B*math.pow(math.sin(phi), 2) + C*math.pow(math.sin(phi), 4) + D*math.pow(math.sin(phi), 6))
    delta_lambda = lambda_ - lambda_zero
    xi_prim = math.atan(math.tan(phi_star) / math.cos(delta_lambda))
    eta_prim = math_atanh(math.cos(phi_star) * math.sin(delta_lambda))
    x = scale * a_roof * (xi_prim +beta1 * math.sin(2.0*xi_prim) * math_cosh(2.0*eta_prim) +beta2 * math.sin(4.0*xi_prim) * math_cosh(4.0*eta_prim) +beta3 * math.sin(6.0*xi_prim) * math_cosh(6.0*eta_prim) +beta4 * math.sin(8.0*xi_prim) * math_cosh(8.0*eta_prim)) + false_northing
    y = scale * a_roof * (eta_prim +beta1 * math.cos(2.0*xi_prim) * math_sinh(2.0*eta_prim) +beta2 * math.cos(4.0*xi_prim) * math_sinh(4.0*eta_prim) +beta3 * math.cos(6.0*xi_prim) * math_sinh(6.0*eta_prim) +beta4 * math.cos(8.0*xi_prim) * math_sinh(8.0*eta_prim)) + false_easting
    x_y[0] = math.round(x * 1000.0) / 1000.0
    x_y[1] = math.round(y * 1000.0) / 1000.0
#  x_y[0] = x
#  x_y[1] = y
    return x_y


# Conversion from grid coordinates to geodetic coordinates.
def grid_to_geodetic(x, y) :
    lat_lon = [0] * 2
    if (central_meridian == None) :
        return lat_lon

    # Prepare ellipsoid-based stuff.
    e2 = flattening * (2.0 - flattening)
    n = flattening / (2.0 - flattening)
    a_roof = axis / (1.0 + n) * (1.0 + n*n/4.0 + n*n*n*n/64.0)
    delta1 = n/2.0 - 2.0*n*n/3.0 + 37.0*n*n*n/96.0 - n*n*n*n/360.0
    delta2 = n*n/48.0 + n*n*n/15.0 - 437.0*n*n*n*n/1440.0
    delta3 = 17.0*n*n*n/480.0 - 37*n*n*n*n/840.0
    delta4 = 4397.0*n*n*n*n/161280.0

    Astar = e2 + e2*e2 + e2*e2*e2 + e2*e2*e2*e2
    Bstar = -(7.0*e2*e2 + 17.0*e2*e2*e2 + 30.0*e2*e2*e2*e2) / 6.0
    Cstar = (224.0*e2*e2*e2 + 889.0*e2*e2*e2*e2) / 120.0
    Dstar = -(4279.0*e2*e2*e2*e2) / 1260.0

    # Convert.
    deg_to_rad = math.pi / 180
    lambda_zero = central_meridian * deg_to_rad
    xi = (x - false_northing) / (scale * a_roof)       
    eta = (y - false_easting) / (scale * a_roof)
    xi_prim = xi - delta1*math.sin(2.0*xi) * math_cosh(2.0*eta) - delta2*math.sin(4.0*xi) * math_cosh(4.0*eta) - delta3*math.sin(6.0*xi) * math_cosh(6.0*eta) - delta4*math.sin(8.0*xi) * math_cosh(8.0*eta)
    eta_prim = eta - delta1*math.cos(2.0*xi) * math_sinh(2.0*eta) - delta2*math.cos(4.0*xi) * math_sinh(4.0*eta) - delta3*math.cos(6.0*xi) * math_sinh(6.0*eta) - delta4*math.cos(8.0*xi) * math_sinh(8.0*eta)
    phi_star = math.asin(math.sin(xi_prim) / math_cosh(eta_prim))
    delta_lambda = math.atan(math_sinh(eta_prim) / math.cos(xi_prim))
    lon_radian = lambda_zero + delta_lambda
    lat_radian = phi_star + math.sin(phi_star) * math.cos(phi_star) * (Astar + Bstar*math.pow(math.sin(phi_star), 2) + Cstar*math.pow(math.sin(phi_star), 4) + Dstar*math.pow(math.sin(phi_star), 6))    
    lat_lon[0] = lat_radian * 180.0 / math.pi
    lat_lon[1] = lon_radian * 180.0 / math.pi
    return lat_lon


# Missing defs in the math library.
def math_sinh(value) :
    return 0.5 * (math.exp(value) - math.exp(-value))

def math_cosh(value) :
    return 0.5 * (math.exp(value) + math.exp(-value))

def math_atanh(value) :
    return 0.5 * math.log((1.0 + value) / (1.0 - value))

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [65]:
key = config["tokens"]["vaginfo"]
urlstr = "http://opendata.linkoping.se/ws_opendata/main.asmx/VagarbeteAlla?CustomKey=" + key

data = {}
with urllib.request.urlopen(urlstr) as url:
    data = xmltodict.parse(url.read().decode())


In [66]:
def timeconv(item):
    start = item['STARTTID'].replace("MAJ","MAY")
    start = start.replace('OKT', 'OCT')
    slut = item['SLUTTID'].replace("MAJ","MAY")
    slut = slut.replace('OKT', 'OCT')
    item['STARTTID'] = start
    item['SLUTTID'] = slut
    return item
data2 = [timeconv(item) for item in data['ResponseVagarbete']['ListaVagarbeten']['Vagarbete'] ]
df = pd.DataFrame.from_dict(data2)
df2 = df.sort_values('IDNR')

In [67]:
def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val,val)

In [68]:
m = Map(center=(58.41, 15.6), zoom=13, basemap=basemaps.OpenStreetMap.Mapnik)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [69]:
df2['SLUTTID'] = pd.to_datetime(df2['SLUTTID'])
df2['STARTTID'] = pd.to_datetime(df2['STARTTID'])
df3 = df2.loc[df2['SLUTTID'] > date.today(),:]
df3

/home/daniel/python_virtualenvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


,IDNR,STARTTID,SLUTTID,PLATS,TFN,BESKRIVNING,TYP,FRAMKOMLIGHET_CYKEL,FRAMKOMLIGHET_BIL,ROADSIGN,LINK_URL,X_SWEREF991500,Y_SWEREF991500
938,1965,2017-08-01,2019-06-01,Kungsgatan 18,2017-2542,Husbyggnation. Kungsgatan enkelriktad mellan R...,Husbyggnation,"0,80","0,80",A20,None,186481.57,6477474.63
939,1965,2017-08-01,2019-04-30,Kungsgatan 18,2017-2542,Husbyggnation. Kungsgatan enkelriktad mellan R...,Husbyggnation,"0,80","0,80",A20,None,186481.57,6477474.63
1010,2017,2017-04-05,2019-04-05,"Björkforsvägen ""ICA""",2017-765,Nybyggnation av hus,Husbyggnation,"0,50","0,50",A40,None,192197.40,6469123.59
1053,2048,2017-08-21,2020-12-31,Gång- ocg cykelbana mellan Rotegatan och Batal...,2017-1893,Gång- och cykelbana avstängd.,Husbyggnation,"0,00","0,00",C2,None,186572.12,6475227.94
1054,2049,2017-08-21,2020-12-31,Garnisonsvägen,2017-1893,Gång- och cykelbana avstängd på ena sidan.,Husbyggnation,"0,00","1,00",C2,None,186400.67,6475171.85
1055,2050,2017-08-21,2020-12-31,Bataljonsvägen,2017-1893,Husbyggnation,Husbyggnation,"0,80","0,70",A20,None,186456.40,6475256.31
1125,2105,2017-09-18,2021-07-01,Kvarteret Eddan,2017-2132,Nybyggnation,Husbyggnation,"0,50","1,00",A20,None,186214.34,6476947.42
1385,2319,2017-12-30,2019-04-30,Industrigatan,2017-2521,Tillfällig in- och utfart från byggarbetsplats...,Husbyggnation,"1,00","1,00",A40,None,184233.22,6477085.76
1397,2329,2018-01-09,2020-01-01,Platensgatan 27,2018-9,Rivning och nyproduktion av hus,Husbyggnation,"0,50","0,50",A20,None,186082.50,6477952.89
1481,2393,2018-02-06,2019-07-21,Tallboda skola,2018-244,Renovering av skola,en avspärrning,"1,00","1,00",A40,None,189741.42,6479245.18


In [71]:
#def get_location(swerefx, swerefy):
#    return [58.41, 15.6]
swedish_params("sweref_99_1500")
def create_marker(datapoint):
    loc = grid_to_geodetic(float(datapoint.Y_SWEREF991500), float(datapoint.X_SWEREF991500))
    roadcondition = ""
    if datapoint.FRAMKOMLIGHET_BIL:
        roadcondition = "Avstängd" if float(datapoint.FRAMKOMLIGHET_BIL.replace(',','.'))<0.1 else "Begränsad"
    htmltext = """
            <div>{}
            <ul class='list-group'>
                <li class='list-group-item'>Loc: {}</li>
                <li class='list-group-item'>Start: {}, Slut: {}</li>
                <li class='list-group-item'>Framkomlighet: {}</li>
            </ul></div>""".format(
        datapoint.BESKRIVNING,
        datapoint.PLATS,
        datapoint.STARTTID,
        datapoint.SLUTTID,
        roadcondition)
    html_widget = HTML(
        value=htmltext,
        placeholder='',
        description=''
    )
    return Marker(location=loc, popup=html_widget)

for item in range(0,len(df3)):
    mark = create_marker(df3.iloc[item,:])
    m += mark
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …